In [5]:
!pip install streamlit

In [14]:
%%writefile app.py
import streamlit as st
from PIL import Image, ExifTags
import numpy as np
import io
from tensorflow.keras.models import load_model

# Load the pre-trained models (assuming models are saved in the same directory)
@st.cache_resource
def load_models():
    reveal_model = load_model('reveal_model_256.h5', compile=False)
    encoder_model = load_model('encoder_model_256.h5', compile=False)
    autoencoder_model = load_model('autoencoder_model_256.h5', compile=False)
    return reveal_model, encoder_model, autoencoder_model

reveal_model, encoder_model, autoencoder_model = load_models()

# Helper function to preprocess images
# Helper function to preprocess images, handling EXIF orientation
def preprocess_image(image, target_size=(256, 256)):
    img = Image.open(image)

    # Handle EXIF orientation
    try:
        for orientation in ExifTags.TAGS.keys():
            if ExifTags.TAGS[orientation] == 'Orientation':
                break
        exif = img._getexif()
        if exif is not None:
            orientation_value = exif.get(orientation, None)
            if orientation_value == 3:
                img = img.rotate(180, expand=True)
            elif orientation_value == 6:
                img = img.rotate(270, expand=True)
            elif orientation_value == 8:
                img = img.rotate(90, expand=True)
    except (AttributeError, KeyError, IndexError):
        # No EXIF or unknown orientation; continue without rotating
        pass

    # Resize and normalize the image
    img = img.resize(target_size)
    img = np.array(img) / 255.0
    img = np.expand_dims(img, axis=0)
    return img


# Helper function for computing the difference
def compute_diff(original, decoded):
    diff = np.abs(decoded - original) * 255 * 0.5  # Enhancing the difference for visibility
    diff = np.clip(diff, 0, 255).astype(np.uint8)
    return diff

# Text steganography helper functions
def message2binary(message):
    if type(message) == str:
        return ''.join([format(byte, "08b") for byte in message.encode('utf-8')])
    elif type(message) == bytes or type(message) == np.ndarray:
        return [format(int(i), "08b") for i in message]
    elif type(message) == int or type(message) == np.uint8:
        return format(message, "08b")
    else:
        raise TypeError("Input type not supported")

def encode_data(img, data):
    img = np.array(img)
    data += '#END#'
    data_binary = message2binary(data)
    data_len = len(data_binary)

    data_index = 0
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            pixel = img[i, j]
            if len(pixel) == 3:
                r, g, b = message2binary(pixel)
            elif len(pixel) == 4:
                r, g, b, _ = message2binary(pixel)
            if data_index < data_len:
                pixel[0] = int(r[:-1] + data_binary[data_index], 2)
                data_index += 1
            if data_index < data_len:
                pixel[1] = int(g[:-1] + data_binary[data_index], 2)
                data_index += 1
            if data_index < data_len:
                pixel[2] = int(b[:-1] + data_binary[data_index], 2)
                data_index += 1
            img[i, j] = pixel
            if data_index >= data_len:
                break
    return Image.fromarray(img.astype('uint8'))

def decode_data(image):
    img = np.array(image)
    binary_data = ""
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            pixel = img[i, j]
            if len(pixel) == 3:
                r, g, b = message2binary(pixel)
            elif len(pixel) == 4:
                r, g, b, _ = message2binary(pixel)
            binary_data += r[-1]
            binary_data += g[-1]
            binary_data += b[-1]
    all_bytes = [binary_data[i:i+8] for i in range(0, len(binary_data), 8)]
    decoded_data = bytearray([int(byte, 2) for byte in all_bytes]).decode('utf-8', errors='ignore')
    end_index = decoded_data.find("#END#")
    if end_index != -1:
        decoded_data = decoded_data[:end_index]
    return decoded_data

# Main Steganography App
st.title("Steganography")

# Create tabs for Image Steganography and Text Steganography
tabs = st.tabs(["Image Steganography", "Text Steganography"])

# --------------- Image Steganography Tab ---------------
with tabs[0]:
    st.header("Image Steganography")

    # Upload cover and secret images
    col1, col2 = st.columns(2)
    with col1:
        cover_image = st.file_uploader("Upload Cover Image", type=["jpg", "jpeg", "png"])
    with col2:
        secret_image = st.file_uploader("Upload Secret Image", type=["jpg", "jpeg", "png"])

    if cover_image and secret_image:
        # Preprocess images
        cover_img = preprocess_image(cover_image)
        secret_img = preprocess_image(secret_image)

        # Display uploaded images
        st.header("Uploaded Images")
        col1, col2 = st.columns(2)
        with col1:
            st.image(np.squeeze(preprocess_image(cover_image)), caption="Cover Image")
        with col2:
            st.image(np.squeeze(preprocess_image(secret_image)), caption="Secret Image")

        # Encode button
        if st.button("Encode Image"):
            encoded_cover = encoder_model.predict([secret_img, cover_img])
            encoded_cover = np.clip(encoded_cover, 0, 1)
            st.header("Encoded Cover Image")
            st.image(np.squeeze(encoded_cover), caption="Encoded Cover Image")
            st.session_state.encoded_cover = encoded_cover
            st.session_state.secret_img = secret_img

        # Decode button
        if st.button("Decode Image"):
            if 'encoded_cover' in st.session_state and 'secret_img' in st.session_state:
                decoded_secret = reveal_model.predict(st.session_state.encoded_cover)
                decoded_secret = np.clip(decoded_secret, 0, 1)
                st.header("Decoded Secret Image")
                st.image(np.squeeze(decoded_secret), caption="Decoded Secret Image")
                diff_secret = compute_diff(st.session_state.secret_img[0], decoded_secret[0])
                st.header("Difference (Secret Image)")
                st.image(diff_secret, caption="Diff Secret Image")
            else:
                st.warning("Please encode the images first.")

# --------------- Text Steganography Tab ---------------
with tabs[1]:
    st.header("Text Steganography")

    operation = st.selectbox("Choose Operation", ["Encode", "Decode"])

    if operation == "Encode":
        data_to_hide = st.text_input("Enter the text to hide:")
        uploaded_image = st.file_uploader("Upload an image", type=["jpg", "jpeg", "png"])

        if uploaded_image and data_to_hide:
            image = Image.open(uploaded_image)
            st.image(image, caption="Uploaded Image", use_column_width=True)

            if st.button("EncodeText"):
                try:
                    encoded_img = encode_data(image, data_to_hide)
                    st.image(encoded_img, caption="Encoded Image", use_column_width=True)
                    buf = io.BytesIO()
                    encoded_img.save(buf, format="PNG")
                    byte_im = buf.getvalue()
                    st.download_button("Download Encoded Image", data=byte_im, file_name="encoded_image.png", mime="image/png")
                except ValueError as e:
                    st.error(str(e))
        elif uploaded_image:
            st.warning("Please enter data to hide.")
        elif data_to_hide:
            st.warning("Please upload an image.")

    elif operation == "Decode":
        uploaded_image = st.file_uploader("Upload an encoded image", type=["jpg", "jpeg", "png"])

        if uploaded_image:
            image = Image.open(uploaded_image)
            st.image(image, caption="Uploaded Image", use_column_width=True)

            if st.button("DecodeText"):
                try:
                    decoded_text = decode_data(image)
                    if decoded_text:
                        st.write("Decoded Text:", decoded_text)
                    else:
                        st.error("No hidden message found.")
                except Exception as e:
                    st.error(str(e))
        else:
            st.warning("Please upload an encoded image.")

Overwriting app.py


In [3]:
!wget -q -O - ipv4.icanhazip.com

34.83.231.107


In [8]:
!streamlit run app.py & npx localtunnel --port 8501

^C
